In [1]:
import common as cm

cm.files_rename('src')
data, labels = cm.load_data('src')
print('Size of Data:', len(data))
print('Size of Label:', len(labels))

Size of Data: 3091
Size of Label: 3091


In [2]:
import numpy as np

dataset = [] # dataset
label = [] # label
for (index, image) in enumerate(data):
    result = cm.preprocess_image(image)
    if (result is None):
        continue
    dataset.append(result)
    label.append(labels[index])

data_size = len(dataset)
data_type = type(dataset)
print(f'''
Data size: {data_size}
Data type: {data_type}
''')


Data size: 2902
Data type: <class 'list'>



In [3]:
from tensorflow.keras.utils import to_categorical

X = np.array(dataset)

print('X shape:', X.shape)

label_set = set(label)
classes = { val: key for (key, val) in enumerate(label_set)}

print(classes)

Y = np.fromiter([classes[y] for y in label], dtype=np.int)
Y = to_categorical(Y)
dense = len(Y[0])
dense

X shape: (2902, 512, 256, 1)
{'80': 0, 'SP': 1, 'NE': 2, 'TC': 3, 'TK': 4, 'TR': 5}


6

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape

(2321, 512, 256, 1)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

# initiate model
model = Sequential()

# add model layers
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(512, 256, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(Flatten())
model.add(Dense(dense, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 509, 253, 32)      544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 254, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 251, 123, 64)      32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 125, 61, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 58, 128)      131200    
_________________________________________________________________
flatten (Flatten)            (None, 905728)            0         
_________________________________________________________________
dense (Dense)                (None, 6)                 5

In [6]:
# compile model using accuracy to measure the performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Epoch 1/3
73/73 [==============================] - 219s 3s/step - loss: 0.5698 - accuracy: 0.7936 - val_loss: 0.1652 - val_accuracy: 0.9656
Epoch 2/3
73/73 [==============================] - 216s 3s/step - loss: 0.0820 - accuracy: 0.9750 - val_loss: 0.1180 - val_accuracy: 0.9776
Epoch 3/3
73/73 [==============================] - 218s 3s/step - loss: 0.0282 - accuracy: 0.9901 - val_loss: 0.0901 - val_accuracy: 0.9931


In [8]:
model.save('saved_models/my_model')

INFO:tensorflow:Assets written to: saved_models/my_model\assets
